<a href="https://colab.research.google.com/github/Asrimathi/code.py/blob/main/CodeAlpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
import json
import time

# Class for each stock in the portfolio
class Stock:
    def __init__(self, symbol, shares, buy_price):
        self.symbol = symbol
        self.shares = shares
        self.buy_price = buy_price

    def get_real_time_price(self):
        """
        Fetch the real-time stock price using Alpha Vantage API.
        """
        API_KEY = 'your_api_key_here'  # Replace with your Alpha Vantage API Key
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={self.symbol}&interval=1min&apikey={API_KEY}'
        response = requests.get(url)
        data = response.json()

        try:
            # Extract the latest stock price from the JSON response
            latest_time = list(data['Time Series (1min)'].keys())[0]
            current_price = float(data['Time Series (1min)'][latest_time]['4. close'])
            return current_price
        except KeyError:
            print(f"Error fetching data for {self.symbol}. Check the symbol or API limit.")
            return None

    def current_value(self):
        """
        Calculate the current value of the stock holdings.
        """
        current_price = self.get_real_time_price()
        if current_price:
            return current_price * self.shares
        else:
            return 0

    def profit_loss(self):
        """
        Calculate the profit or loss based on the current stock price and the buy price.
        """
        current_price = self.get_real_time_price()
        if current_price:
            return (current_price - self.buy_price) * self.shares
        else:
            return 0

# Portfolio class to manage multiple stocks
class Portfolio:
    def __init__(self):
        self.stocks = {}

    def add_stock(self, symbol, shares, buy_price):
        if symbol in self.stocks:
            print(f"Stock {symbol} already exists. Updating shares.")
            self.stocks[symbol].shares += shares
        else:
            self.stocks[symbol] = Stock(symbol, shares, buy_price)
        print(f"Added {shares} shares of {symbol} at ${buy_price}.")

    def remove_stock(self, symbol):
        if symbol in self.stocks:
            del self.stocks[symbol]
            print(f"Removed {symbol} from portfolio.")
        else:
            print(f"{symbol} not found in portfolio.")

    def view_portfolio(self):
        """
        Display the entire portfolio with real-time stock data and profit/loss.
        """
        total_value = 0
        total_profit_loss = 0
        print(f"{'Symbol':<10} {'Shares':<10} {'Buy Price':<10} {'Current Price':<15} {'Value':<10} {'Profit/Loss'}")
        for stock in self.stocks.values():
            current_price = stock.get_real_time_price()
            if current_price:
                value = stock.current_value()
                profit_loss = stock.profit_loss()
                total_value += value
                total_profit_loss += profit_loss
                print(f"{stock.symbol:<10} {stock.shares:<10} {stock.buy_price:<10} {current_price:<15.2f} {value:<10.2f} {profit_loss:<10.2f}")
            else:
                print(f"{stock.symbol}: Unable to fetch real-time data.")
        print(f"\nTotal Portfolio Value: ${total_value:.2f}")
        print(f"Total Profit/Loss: ${total_profit_loss:.2f}")

# Main program execution
def main():
    portfolio = Portfolio()

    while True:
        print("\n1. Add Stock\n2. Remove Stock\n3. View Portfolio\n4. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            symbol = input("Enter stock symbol (e.g., AAPL): ").upper()
            shares = int(input(f"Enter number of shares for {symbol}: "))
            buy_price = float(input(f"Enter the buy price for {symbol}: "))
            portfolio.add_stock(symbol, shares, buy_price)

        elif choice == '2':
            symbol = input("Enter stock symbol to remove: ").upper()
            portfolio.remove_stock(symbol)

        elif choice == '3':
            portfolio.view_portfolio()

        elif choice == '4':
            print("Exiting program...")
            break

        else:
            print("Invalid choice! Please try again.")

if __name__ == '__main__':
    main()